# Tic-Tac-Toe (Events + Observers)

This notebook uses event-style inputs with a `events` namespace.
Human plays X, AI plays O. The widget emits outputs, and Python responds by issuing events.

In [1]:
import time
import vibe_widget as vw
import math
import random


# Create move prediction

In [2]:
def check_winner(board):
    # Winning combinations indices
    wins = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8), # Rows
        (0, 3, 6), (1, 4, 7), (2, 5, 8), # Cols
        (0, 4, 8), (2, 4, 6)             # Diagonals
    ]
    for a, b, c in wins:
        if board[a] == board[b] == board[c] and board[a] != 'b':
            return board[a]
    if 'b' not in board:
        return 'tie'
    return None

def minimax(board, depth, is_maximizing):
    result = check_winner(board)
    if result == 'o': return 10 - depth
    if result == 'x': return -10 + depth
    if result == 'tie': return 0

    if is_maximizing:
        best_score = -math.inf
        for i in range(9):
            if board[i] == 'b':
                board[i] = 'o'
                score = minimax(board, depth + 1, False)
                board[i] = 'b' # Backtrack
                best_score = max(score, best_score)
        return best_score
    else:
        best_score = math.inf
        for i in range(9):
            if board[i] == 'b':
                board[i] = 'x'
                score = minimax(board, depth + 1, True)
                board[i] = 'b' # Backtrack
                best_score = min(score, best_score)
        return best_score

# fn we'll trigger with an observer 
def pick_best_move(board_list):
    """
    Uses Minimax to find the optimal move. 
    Returns the index of the best move.
    """
    best_score = -math.inf
    best_moves = []
    
    # Check for immediate empty spots first to save time on empty board
    empty_spots = [i for i, x in enumerate(board_list) if x == 'b']
    
    # Optimization: If board is empty, pick center or corner to save recursion depth
    if len(empty_spots) == 9:
        return 4 # Center is usually best opener
    if len(empty_spots) == 8 and board_list[4] == 'b':
        return 4 # Take center if opponent didn't

    for i in empty_spots:
        # Try the move
        board_list[i] = 'o'
        score = minimax(board_list, 0, False)
        board_list[i] = 'b' # Undo the move

        if score > best_score:
            best_score = score
            best_moves = [i]
        elif score == best_score:
            best_moves.append(i)
            
    if best_moves:
        return random.choice(best_moves) # Randomize if multiple moves are equally perfect
    return None

## Observe outputs and issue events

In [ ]:
                                                          
game_board = vw.create(                                    
      """Interactive Tic-Tac-Toe game board                                
      - Human plays X, AI plays O      
      - Click cells to make moves      
      """,                                                   
      outputs=vw.outputs(                                    
          board_state="9-element array of 'x', 'o', or 'b'", 
          game_over="boolean",                               
          current_turn="'x' or 'o'"                          
      ),                                                     
      actions=vw.actions(                                    
          ai_move=vw.action(                                 
              "AI move at index 0-8 (row-major)",            
              params={"index": "0-8 row-major"}              
          )                                                  
      ),                                                     
      cache=False                                            
)               

game_board


def on_turn_change(event):
    print('on_turn_change', event)
    if event.new != "o":
        return

    # Allow the frontend to finish updating its state.
    time.sleep(0.1)

    board_state = game_board.outputs.board_state.value
    if not board_state or game_board.outputs.game_over.value:
        return

    if isinstance(board_state, str):
        import ast
        board_state = ast.literal_eval(board_state)

    board_list = list(board_state)
    if len(board_list) != 9:
        return

    move_index = pick_best_move(board_list) # can be replaced with a neural net
    if move_index is None:
        return

    game_board.actions.ai_move(index=move_index)


game_board.outputs.current_turn.observe(on_turn_change)


VibeWidget_2113471984471449147(audit_state={'status': 'idle', 'response': {}, 'error': '', 'request': {}, 'app...

In [7]:
game_board.code

'import React from "https://esm.sh/react@18";\n\nexport const Square = ({ value, onClick, disabled, html }) => {\n  const color = value === \'x\' ? \'#3b82f6\' : value === \'o\' ? \'#ef4444\' : \'transparent\';\n  return html`\n    <button \n      onClick=${onClick}\n      disabled=${disabled || value !== \'b\'}\n      style=${{\n        width: \'100px\',\n        height: \'100px\',\n        fontSize: \'3rem\',\n        fontWeight: \'bold\',\n        display: \'flex\',\n        alignItems: \'center\',\n        justifyContent: \'center\',\n        backgroundColor: \'#ffffff\',\n        border: \'2px solid #e5e7eb\',\n        borderRadius: \'8px\',\n        cursor: value === \'b\' && !disabled ? \'pointer\' : \'default\',\n        color: color,\n        transition: \'all 0.2s ease\',\n        boxShadow: value === \'b\' ? \'0 1px 3px rgba(0,0,0,0.1)\' : \'none\'\n      }}\n    >\n      ${value === \'x\' ? \'X\' : value === \'o\' ? \'O\' : \'\'}\n    </button>\n  `;\n};\n\nexport const Gam

In [4]:
import vibe_widget as vw
import numpy as np

# Combined terrain editor + 3D viewer with erosion timeline
terrain = vw.create(
    """Split-view terrain editor with erosion timeline
    LEFT PANEL (40% width): 2D canvas painter
    - 64x64 heightmap, grayscale display (black=0, white=1)
    - Mouse drag to paint terrain height
    - Brush size slider (1-10 cells)
    - Brush height slider (0-1)
    - "Run Erosion" button that pulses run_erosion output (0.05 erosion strength)
    - "Clear" button to reset to flat terrain (0.3 everywhere)
    
    RIGHT PANEL (60% width): 3D Three.js viewer
    - Mesh generated from heightmap (height scaled by 10)
    - Water plane at y=2 (semi-transparent blue)
    - Orbit controls for camera
    - Vertex colors: blue (<0.2), green (0.2-0.5), brown (0.5-0.8), white (>0.8)
    
    BOTTOM (full width): Erosion timeline controls
    - Only visible after erosion stages received
    - Slider labeled "Erosion Stage" from 1-7
    - Labels showing "Stage X / 7" 
    - Optional: play button to auto-animate through stages
    
    Both views share same heightmap state - painting updates 3D in real-time.
    When erosion stages received, slider controls which stage displays.
    Painting new terrain hides the timeline (clears stages).
    
    Can receive erosion_stages via set_erosion_stages action.
    """,
    outputs=vw.outputs(
        heightmap="64x64 nested array of floats 0-1",
        run_erosion="boolean pulse when erosion button clicked"
    ),
    actions=vw.actions(
        set_erosion_stages=vw.action(
            "Set array of 7 heightmap snapshots from erosion simulation",
            params={"stages": "array of 7 heightmaps, each 64x64 nested array of floats 0-1"}
        )
    ),
)

# Erosion simulation with checkpoints
def erode_with_stages(heightmap, num_stages=7, total_iterations=200, erosion_strength=0.1):
    """Hydraulic erosion simulation returning intermediate stages"""
    h = np.array(heightmap, dtype=np.float32)
    stages = [h.copy().tolist()]  # Stage 0: original
    
    iterations_per_stage = total_iterations // (num_stages - 1)
    
    for stage in range(1, num_stages):
        for _ in range(iterations_per_stage):
            pad_h = np.pad(h, 1, mode='edge')
            neighbors = [
                pad_h[:-2, 1:-1],  # up
                pad_h[2:, 1:-1],   # down
                pad_h[1:-1, :-2],  # left
                pad_h[1:-1, 2:],   # right
            ]
            
            min_neighbor = np.minimum.reduce(neighbors)
            diff = h - min_neighbor
            
            erosion = erosion_strength * np.maximum(diff, 0)
            h -= erosion
            h += 0.3 * erosion_strength * (np.mean(neighbors, axis=0) - h)
            h = np.clip(h, 0, 1)
        
        stages.append(h.copy().tolist())
        print(f"Stage {stage}/{num_stages - 1} complete")
    
    return stages

# Wire up erosion
def on_erosion_click(change):
    if not change.new:
        return
    
    current = terrain.outputs.heightmap.value
    if current is None:
        return
    
    print("Running erosion simulation...")
    stages = erode_with_stages(current, num_stages=7, total_iterations=210)
    terrain.actions.set_erosion_stages(stages=stages)
    print(f"Done! {len(stages)} stages sent to viewer.")

terrain.outputs.run_erosion.observe(on_erosion_click)

terrain

VibeWidget_6785079815008291887(audit_state={'status': 'idle', 'response': {}, 'error': '', 'request': {}, 'app...

VibeWidget_6785079815008291887(audit_state={'status': 'idle', 'response': {}, 'error': '', 'request': {}, 'app...

In [1]:
  import vibe_widget as vw
  w = vw.create("input mismatch",
  inputs=vw.inputs(data=[1,2,3]), display=True,
  cache=False)
  # Overwrite code to assume data is an object
  w.code = '''
  export default function Widget({ model, html }) {
    const data = model.get("data");
    return html`<div>${data.notAField}</div>`;
  } '''

  

VibeWidget(audit_state={'status': 'idle', 'response': {}, 'error': '', 'request': {}, 'apply': {'status': 'idl...

In [2]:
print(vw.__file__)

/Users/dwootton/Projects/vibe-widgets-sundai/src/vibe_widget/__init__.py


In [5]:
def show_debug(widget):
      print("widget_error:", getattr(widget,
  "widget_error", None))
      print("error_message:", getattr(widget,
  "error_message", None))
      print("widget_logs (last 3):", (getattr(widget,
  "widget_logs", []) or [])[-3:])

show_debug(w)

widget_error: 
error_message: 
widget_logs (last 3): []


In [3]:
  import vibe_widget as vw
  w = vw.create("check bundle", display=False,
  cache=False)
  print("widget_error in bundle:", "widget_error" in w._esm)
  print("widget_logs in bundle:", "widget_logs" in w._esm)

widget_error in bundle: True
widget_logs in bundle: True


In [1]:
  import vibe_widget as vw
  w = vw.create("banner test", display=True, cache=False)
  w.widget_error = "Manual error banner test"
  w.widget_logs = [{"timestamp": 0, "message": "manual log", "level": "info", "source": "python"}]

VibeWidget(audit_state={'status': 'idle', 'response': {}, 'error': '', 'request': {}, 'apply': {'status': 'idl...

Widget runtime error:
Manual error banner test


In [2]:
w = vw.create("runtime crash", display=True, cache=False)
w.code = "export default function Widget(){ throw new Error('boom'); }"

VibeWidget(audit_state={'status': 'idle', 'response': {}, 'error': '', 'request': {}, 'apply': {'status': 'idl...

In [4]:
  w = vw.create("log test", display=True, cache=False)
  w.code = '''
  export default function Widget({ html }) {
    console.log("hello from widget");
    return html`<div>ok</div>`;
  }'''

VibeWidget(audit_state={'status': 'idle', 'response': {}, 'error': '', 'request': {}, 'apply': {'status': 'idl...